# Class 31: Intro to NLP

1. say hello on zoom
1. share a sentence on the doc linked on prismia

In [1]:
import numpy as np

In [2]:
# %load http://drsmb.co/310
def classification_confint(acc, n):
    '''
    Compute the 95% confidence interval for a classification problem.
      acc -- classification accuracy
      n   -- number of observations used to compute the accuracy
    Returns a tuple (lb,ub)
    '''
    interval = 1.96*np.sqrt(acc*(1-acc)/n)
    lb = max(0, acc - interval)
    ub = min(1.0, acc + interval)
    return (lb,ub)

In [3]:
classification_confint(.78,50)

(0.6651767828355258, 0.8948232171644742)

In [4]:
classification_confint(.9,50)

(0.816844242532462, 0.983155757467538)

In [5]:
N =200
classification_confint(.9,N)

(0.8584221212662311, 0.941577878733769)

In [6]:
classification_confint(.78,N)

(0.722588391417763, 0.8374116085822371)

## Natural Language Processing

### represention

- vector
- bag of words


terms:

- document: unit of text we're analyzing (one sample)
- token: word
- stop words: no meaning, we don't need them

In [7]:
# %load http://drsmb.co/310
text = {
'Demeus Alves':'Hope everybody is staying safe',
'Ryan Booth':'The power is out where I live, might be forced to leave soon',
'Brianna MacDonald':'Rainy days',
'Jair Delgado':'Can not wait for lunch... hungry',
'Shawn Vincent':'I am excited for Thaksgiving',
'Jacob Afonso':'Short weeks are the best!',
'Ryan Buquicchio':'The sentence is sentence. (Best sentence ever)',
'Nick McCaffery':'Very windy today',
'David Perrone':'this is a sentence',
'Masoud':'It is rainy here. What about there?',
'Rony Lopes':'I get to relax later this week',
'Patrick Dowd':'It is cold out today',
'Ruifang Kuang':'Happy Thanksgiving!',
}

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances

In [9]:
text['Demeus Alves']

'Hope everybody is staying safe'

In [10]:
s1 = text['Demeus Alves']

In [11]:
s1

'Hope everybody is staying safe'

In [12]:
counts = CountVectorizer()

In [13]:
counts.fit_transform([s1])

<1x5 sparse matrix of type '<class 'numpy.int64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [14]:
counts.vocabulary_

{'hope': 1, 'everybody': 0, 'is': 2, 'staying': 4, 'safe': 3}

In [15]:
counts.fit_transform(text.values())

<13x49 sparse matrix of type '<class 'numpy.int64'>'
	with 65 stored elements in Compressed Sparse Row format>

In [16]:
counts.vocabulary_

{'hope': 16,
 'everybody': 9,
 'is': 18,
 'staying': 34,
 'safe': 30,
 'the': 37,
 'power': 27,
 'out': 26,
 'where': 47,
 'live': 22,
 'might': 24,
 'be': 3,
 'forced': 12,
 'to': 40,
 'leave': 21,
 'soon': 33,
 'rainy': 28,
 'days': 7,
 'can': 5,
 'not': 25,
 'wait': 43,
 'for': 11,
 'lunch': 23,
 'hungry': 17,
 'am': 1,
 'excited': 10,
 'thaksgiving': 35,
 'short': 32,
 'weeks': 45,
 'are': 2,
 'best': 4,
 'sentence': 31,
 'ever': 8,
 'very': 42,
 'windy': 48,
 'today': 41,
 'this': 39,
 'it': 19,
 'here': 15,
 'what': 46,
 'about': 0,
 'there': 38,
 'get': 13,
 'relax': 29,
 'later': 20,
 'week': 44,
 'cold': 6,
 'happy': 14,
 'thanksgiving': 36}

In [17]:
mat = counts.fit_transform(text.values()).toarray()
mat

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
        1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0

In [18]:
import pandas as pd

In [19]:
counts.get_feature_names()

['about',
 'am',
 'are',
 'be',
 'best',
 'can',
 'cold',
 'days',
 'ever',
 'everybody',
 'excited',
 'for',
 'forced',
 'get',
 'happy',
 'here',
 'hope',
 'hungry',
 'is',
 'it',
 'later',
 'leave',
 'live',
 'lunch',
 'might',
 'not',
 'out',
 'power',
 'rainy',
 'relax',
 'safe',
 'sentence',
 'short',
 'soon',
 'staying',
 'thaksgiving',
 'thanksgiving',
 'the',
 'there',
 'this',
 'to',
 'today',
 'very',
 'wait',
 'week',
 'weeks',
 'what',
 'where',
 'windy']

In [20]:
text_df = pd.DataFrame(data=mat, index = text.keys(), columns=counts.get_feature_names() )
text_df

,about,am,are,be,best,can,cold,days,ever,everybody,...,this,to,today,very,wait,week,weeks,what,where,windy
Demeus Alves,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Ryan Booth,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
Brianna MacDonald,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Jair Delgado,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
Shawn Vincent,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Jacob Afonso,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Ryan Buquicchio,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Nick McCaffery,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,1
David Perrone,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Masoud,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [21]:
euclidean_distances(text_df)

array([[0.        , 3.87298335, 2.64575131, 3.31662479, 3.        ,
        3.16227766, 4.        , 2.82842712, 2.44948974, 3.16227766,
        3.31662479, 2.82842712, 2.64575131],
       [3.87298335, 0.        , 3.74165739, 4.24264069, 4.        ,
        3.87298335, 4.58257569, 3.87298335, 3.60555128, 4.12310563,
        4.        , 3.60555128, 3.74165739],
       [2.64575131, 3.74165739, 0.        , 2.82842712, 2.44948974,
        2.64575131, 3.87298335, 2.23606798, 2.23606798, 2.64575131,
        2.82842712, 2.64575131, 2.        ],
       [3.31662479, 4.24264069, 2.82842712, 0.        , 2.82842712,
        3.31662479, 4.35889894, 3.        , 3.        , 3.60555128,
        3.46410162, 3.31662479, 2.82842712],
       [3.        , 4.        , 2.44948974, 2.82842712, 0.        ,
        3.        , 4.12310563, 2.64575131, 2.64575131, 3.31662479,
        3.16227766, 3.        , 2.44948974],
       [3.16227766, 3.87298335, 2.64575131, 3.31662479, 3.        ,
        0.        , 3.74165

In [22]:
dist_df = pd.DataFrame(data = euclidean_distances(text_df), 
                       index=  text.keys(), columns= text.keys())
dist_df

,Demeus Alves,Ryan Booth,Brianna MacDonald,Jair Delgado,Shawn Vincent,Jacob Afonso,Ryan Buquicchio,Nick McCaffery,David Perrone,Masoud,Rony Lopes,Patrick Dowd,Ruifang Kuang
Demeus Alves,0.000000,3.872983,2.645751,3.316625,3.000000,3.162278,4.000000,2.828427,2.449490,3.162278,3.316625,2.828427,2.645751
Ryan Booth,3.872983,0.000000,3.741657,4.242641,4.000000,3.872983,4.582576,3.872983,3.605551,4.123106,4.000000,3.605551,3.741657
Brianna MacDonald,2.645751,3.741657,0.000000,2.828427,2.449490,2.645751,3.872983,2.236068,2.236068,2.645751,2.828427,2.645751,2.000000
Jair Delgado,3.316625,4.242641,2.828427,0.000000,2.828427,3.316625,4.358899,3.000000,3.000000,3.605551,3.464102,3.316625,2.828427
Shawn Vincent,3.000000,4.000000,2.449490,2.828427,0.000000,3.000000,4.123106,2.645751,2.645751,3.316625,3.162278,3.000000,2.449490
Jacob Afonso,3.162278,3.872983,2.645751,3.316625,3.000000,0.000000,3.741657,2.828427,2.828427,3.464102,3.316625,3.162278,2.645751
Ryan Buquicchio,4.000000,4.582576,3.872983,4.358899,4.123106,3.741657,0.000000,4.000000,2.828427,4.242641,4.358899,4.000000,3.872983
Nick McCaffery,2.828427,3.872983,2.236068,3.000000,2.645751,2.828427,4.000000,0.000000,2.449490,3.162278,3.000000,2.449490,2.236068
David Perrone,2.449490,3.605551,2.236068,3.000000,2.645751,2.828427,2.828427,2.449490,0.000000,2.828427,2.645751,2.449490,2.236068
Masoud,3.162278,4.123106,2.645751,3.605551,3.316625,3.464102,4.242641,3.162278,2.828427,0.000000,3.605551,2.828427,3.000000


In [23]:
dist_df['Masoud'].min()

0.0

In [24]:
dist_df['Masoud'].drop('Masoud')

Demeus Alves         3.162278
Ryan Booth           4.123106
Brianna MacDonald    2.645751
Jair Delgado         3.605551
Shawn Vincent        3.316625
Jacob Afonso         3.464102
Ryan Buquicchio      4.242641
Nick McCaffery       3.162278
David Perrone        2.828427
Rony Lopes           3.605551
Patrick Dowd         2.828427
Ruifang Kuang        3.000000
Name: Masoud, dtype: float64

In [25]:
dist_df['Masoud'].drop('Masoud').min()

2.6457513110645907

In [26]:
dist_df['Masoud'].drop('Masoud').idxmin()

'Brianna MacDonald'